<a href="https://colab.research.google.com/github/falakchhaya/Demo/blob/master/KiteConnect_HistoricalData_Concise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
######## These are the variables that you need to set.

## This is what you send me every morning. Needs to be updated everyday after 7:30 AM
accessToken = "TOPJzZ7CwyzBL0xbMxxoCPm4o74ilHEK" 

# Candle timeframe
interval_at_which_data_required = '5minute' 

# All possible values for candle timeframe
# 'minute', '3minute', '5minute', '10minute', '15minute', '30minute','60minute', 'day'

# Path to the folder in your Google Drive where you want to save the output
pathToFolder = '/content/drive/MyDrive/KiteConnect/'


In [ ]:
!pip install kiteconnect

In [3]:
# Fetching some important libraries
from datetime import datetime, timedelta
from kiteconnect import KiteConnect 
import pandas as pd

In [4]:
# DO NOT SHARE THESE CODES !
apiKey="apec8b5w2brxbydr"
apiSecret="eizb42d3o0orsi7k6ecfivdxm5axk4yl"

# Setting credentials for KiteConnect
kite = KiteConnect(api_key=apiKey)
kite.set_access_token(accessToken)
# print(kite.login_url())


# Let's check the connection by fetching some example data

In [ ]:
# Quick connection check

token = 260105 #136247044  #128008708
toDate = datetime.now() 
fromDate = toDate-timedelta(days=10)
print(fromDate, ' to ', toDate)

interval='3minute'
oi=0

data = kite.historical_data(token,fromDate, toDate, interval,oi)
df = pd.DataFrame(data)
print(df.shape)
print(df.head(10))

# If there is no error below and you can see some data, it means the connection is working fine.


## Kite Connect connection is working fine. Let's get nifty-50 names and tokens. For that, let's connect to Google Drive.


In [6]:
# One has to authorize access to Google drive once.
# Once every day, click on the link below, copy the code from there, past it here and press enter.
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# This file should exist in your Google Sheets. 
# It would contain the instrument token of desired scripts.
# Ideally this file contains following columns, 'Company Name', 'Symbol', 'InstrumentToken'
 
worksheet = gc.open('Nifty50_withTokens').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

pf = pd.DataFrame.from_records(rows)

pf.columns = pf.iloc[0]
pf.drop(index=0, inplace=True)
pf.head() #Let's see how this data from google sheet looks like

## Connection works fine, nifty-50 symbols are fetched. Main task now.

In [8]:
print("Number of scripts/instruments : {}".format(pf.shape[0]))

Number of scripts/instruments : 52


In [ ]:
import time


overViewData = pd.DataFrame(columns=['token', 'label', 'numOfRows'])

for i in range(pf.shape[0]): 

  companyName = pf.iloc[i]['Company Name']
  symbol = pf.iloc[i]['Symbol']
  token = pf.iloc[i]['InstrumentToken']
  interval = interval_at_which_data_required
  d = 20   # number of days for which data is to be fetched at once.
  failCounter = 0
  zeroDataCounter = 0

  toDate = datetime.now()
  fromDate = toDate-timedelta(days=d)

  df = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])  
  
  print("\n")
  print(str(i+1), symbol)  

  for i in range(0,200):  # outer loop
    
    try:
      data = kite.historical_data(token,fromDate, toDate, interval,oi)
    
      df_temp = pd.DataFrame(data)
      df = pd.concat([df,df_temp], ignore_index=True)
      data.clear()

      if df_temp.shape[0] == 0:
        print("Counter No {}: zero data".format(i+1))
        zeroDataCounter +=1      

      del df_temp 
      toDate = fromDate
      fromDate = toDate-timedelta(days=d)
      time.sleep(1)

      if zeroDataCounter >15:
        print("Counter No {}".format(i+1))
        print("{}: No data for more than 25 rows... Breaking...".format(symbol))
        break

    except:
      print("Counter No {}".format(i+1))
      print("error, from {} to {}".format(fromDate, toDate))
      failCounter += 1      
      if failCounter>10:
        print('{} encounterd error >10 times... breaking'.format(symbol))
        break
      time.sleep(2)

    


  df = df.sort_values(by='date').reset_index(drop=True)
  suffix = '_' + interval_at_which_data_required + '.csv'
  fileName = pathToFolder + str(symbol) + suffix
  df.to_csv(fileName)
  print(df.shape)
  new_row = {'token':token , 'label': symbol, 'numOfRows':df.shape[0]}
  overViewData = overViewData.append(new_row, ignore_index=True)
  del df


# 

In [ ]:

# This is for saving an overview, i.e. how many rows are there in each script.
fileName2 = pathToFolder + '01_Overview_' + interval_at_which_data_required + '.csv'
overViewData.to_csv(fileName2)
overViewData